In [1]:
import tensorflow as tf
import json
import numpy as np

In [19]:
features = np.array(["nodes_used", "processors_used", "mem_used", "vmem_used", "interactive"])
NUM_TRAIN_EXAMPLES = 5000
NUM_TEST_EXAMPLES = 100
X_train, X_test = {}, {}
y_train, y_test = np.zeros(NUM_TRAIN_EXAMPLES, dtype=np.int), np.zeros(NUM_TEST_EXAMPLES, dtype=np.int)
my_feature_columns = []

for key in features:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
estimator = tf.estimator.DNNClassifier(
                            feature_columns=my_feature_columns,
                            hidden_units=[10,10],
                            n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/dr/9dqt0xgd2ng_4l3n_n8z9v9h0000gn/T/tmpivoo084v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1108802b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
def train_input_fn(features, labels):
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(10)

def eval_input_fn(features, labels):
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
        
    return tf.data.Dataset.from_tensor_slices(inputs).batch(10)
      
def create_feature_vectors(X, features, obj, iteration):
    for ind, feature in enumerate(features):
        if feature not in X:
            X[feature] = np.zeros(NUM_TRAIN_EXAMPLES)
            X[feature][0] = float(obj["job"][feature])
        else:
            try:
                X[feature][iterations] = float(obj["job"][feature])
            except TypeError:
                X[feature][iterations] = 1.0        

def create_labels(y, obj, iteration):
        if obj["job"]["queue"] == "batch":
            y[iterations] = 1
        else:
            y[iterations] = 0
    
    
with open("10k.anon.json") as data: 
    iterations = 0
    for line in data:
        if iterations < NUM_TRAIN_EXAMPLES:
            obj = json.loads(line)
    
            for ind, feature in enumerate(features):
                if feature not in X_train:
                    X_train[feature] = np.zeros(NUM_TRAIN_EXAMPLES)
                    X_train[feature][0] = float(obj["job"][feature])
                else:
                    try:
                        X_train[feature][iterations] = float(obj["job"][feature])
                    except TypeError:
                        X_train[feature][iterations] = 1.0

            true_label = obj["job"]["queue"] 
            if true_label == "debug":
                y_train[iterations] = 0
            elif true_label == "batch":
                y_train[iterations] = 1
            elif true_label == "short":
                y_train[iterations] = 2
            else:
                y_train[iterations] = 3
                
            #create_feature_vectors(X_train, features, obj, iterations)
            #create_labels(y_train, obj, iterations)
            
            iterations += 1
        elif iterations < NUM_TRAIN_EXAMPLES + NUM_TEST_EXAMPLES:
            for ind, feature in enumerate(features):
                if feature not in X_test:
                    X_test[feature] = np.zeros(NUM_TEST_EXAMPLES)
                    X_test[feature][0] = float(obj["job"][feature])
                else:
                    X_test[feature][iterations-NUM_TRAIN_EXAMPLES] = float(obj["job"][feature])

                    
            true_label = obj["job"]["queue"]
            if true_label == "debug":
                y_test[iterations-NUM_TRAIN_EXAMPLES] = 0
            elif true_label == "batch":
                y_test[iterations-NUM_TRAIN_EXAMPLES] = 1
            elif true_label == "short":
                y_test[iterations-NUM_TRAIN_EXAMPLES] = 2
            else:
                y_test[iterations-NUM_TRAIN_EXAMPLES] = 3
                

            #create_feature_vectors(X_test, features, obj, iterations)    
            #create_labels(y_test, obj, iterations)    
            
            iterations += 1
        else:
            break
        
#print(X_train)
#print(y_train)
            


In [21]:
estimator.train(
                input_fn=lambda:train_input_fn(X_train, y_train),
                steps=5)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/dr/9dqt0xgd2ng_4l3n_n8z9v9h0000gn/T/tmpivoo084v/model.ckpt.
INFO:tensorflow:loss = 2.52004e+06, step = 1
INFO:tensorflow:Saving checkpoints for 5 into /var/folders/dr/9dqt0xgd2ng_4l3n_n8z9v9h0000gn/T/tmpivoo084v/model.ckpt.
INFO:tensorflow:Loss for final step: 7.65357.


In [22]:
eval_result = estimator.evaluate(
    input_fn=lambda:eval_input_fn(X_test, y_test)
    )

INFO:tensorflow:Starting evaluation at 2018-04-08-21:26:09
INFO:tensorflow:Restoring parameters from /var/folders/dr/9dqt0xgd2ng_4l3n_n8z9v9h0000gn/T/tmpivoo084v/model.ckpt-5
INFO:tensorflow:Finished evaluation at 2018-04-08-21:26:09
INFO:tensorflow:Saving dict for global step 5: accuracy = 1.0, average_loss = 1.02753, global_step = 5, loss = 10.2753


In [36]:
X_predict = {}
NUM_PREDICT_EXAMPLES = 5

with open("10k.anon.json") as data: 
    for ind, line in enumerate(data):
        if ind > 6000 and ind < 6005:
            for ind, feature in enumerate(features):
                if feature not in X_predict:
                    X_predict[feature] = np.zeros(NUM_PREDICT_EXAMPLES)
                    X_predict[feature][0] = float(obj["job"][feature])
                else:
                    X_predict[feature][ind] = float(obj["job"][feature])
                    
            print(obj["job"]["queue"])
        elif ind > 6004:
            break

            
print(X_predict)

batch
batch
batch
batch
{'nodes_used': array([ 1.,  0.,  0.,  0.,  0.]), 'processors_used': array([ 1.,  1.,  0.,  0.,  0.]), 'mem_used': array([ 0.,  0.,  0.,  0.,  0.]), 'vmem_used': array([ 0.,  0.,  0.,  0.,  0.]), 'interactive': array([ 0.,  0.,  0.,  0.,  0.])}


In [35]:
pred = estimator.predict(input_fn=lambda:eval_input_fn(X_predict, labels=None))
print(list(pred))

INFO:tensorflow:Restoring parameters from /var/folders/dr/9dqt0xgd2ng_4l3n_n8z9v9h0000gn/T/tmpivoo084v/model.ckpt-5
[{'logits': array([-0.31683797,  0.31001115, -0.1707429 , -0.13384569], dtype=float32), 'probabilities': array([ 0.19121136,  0.35789099,  0.22129008,  0.22960757], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object)}, {'logits': array([-0.16307461,  0.20333187, -0.14378788, -0.21046299], dtype=float32), 'probabilities': array([ 0.226463  ,  0.32668227,  0.23087312,  0.21598159], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object)}, {'logits': array([-0.21756758,  0.21432582, -0.13004884, -0.20768178], dtype=float32), 'probabilities': array([ 0.21544442,  0.33182123,  0.23514955,  0.21758482], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object)}, {'logits': array([-0.21756758,  0.21432582, -0.13004884, -0.20768178], dtype=float32), 'probabilities': array([ 0.21544442,  0.33182123,  0.23514